In [ ]:
# Đo thời gian xử lý
start_time = time.time()

# Đọc dữ liệu sponsor với columns cần thiết
print("Đọc dữ liệu sponsor...")
sponsor_df = pd.read_parquet(
    sponsor_dataset, 
    columns=['videoID', 'startTime', 'endTime', 'votes', 'locked', 
             'hidden', 'shadowHidden', 'reputation', 'timeSubmitted', 'category', 'actionType']
)

print(f"Kích thước ban đầu: {sponsor_df.shape}")
print(f"Số lượng video duy nhất: {sponsor_df['videoID'].nunique()}")
print(f"Thời gian đọc dữ liệu: {time.time() - start_time:.2f} giây")

## 2. Tối ưu hóa lọc dữ liệu SponsorBlock

Chúng ta sẽ áp dụng các bước lọc hiệu quả sau theo thứ tự để nhanh chóng loại bỏ các segment không cần thiết:
1. Loại bỏ các segment bị ẩn (hidden, shadowHidden)
2. Loại bỏ các segment không hợp lệ (startTime, endTime)
3. Loại bỏ các video có quá nhiều segment
4. Lọc segment chất lượng cao (votes, locked, reputation)

In [ ]:
def optimize_sponsorblock_filtering():
    """
    Tối ưu hóa việc lọc dữ liệu SponsorBlock bằng cách ưu tiên các bước lọc hiệu quả
    """
    global sponsor_df
    step_time = time.time()
    
    # 1. Loại bỏ các segment bị ẩn (hidden, shadowHidden) - Lọc nhanh
    sponsor_df = sponsor_df[(sponsor_df['hidden'] == 0) & (sponsor_df['shadowHidden'] == 0)]
    print(f"Sau khi loại bỏ hidden/shadowHidden: {sponsor_df.shape}, Thời gian: {time.time() - step_time:.2f}s")
    step_time = time.time()
    
    # 2. Loại bỏ các segment không hợp lệ (startTime, endTime) - Lọc nhanh
    sponsor_df = sponsor_df[(sponsor_df['startTime'] >= 0) & 
                           (sponsor_df['endTime'] > sponsor_df['startTime']) &
                           (sponsor_df['endTime'] - sponsor_df['startTime'] < 30 * 60)]  # < 30 phút
    print(f"Sau khi loại bỏ thời gian không hợp lệ: {sponsor_df.shape}, Thời gian: {time.time() - step_time:.2f}s")
    step_time = time.time()
    
    # 3. Loại bỏ các video có quá nhiều segment (rất có thể là dữ liệu nhiễu)
    video_counts = sponsor_df['videoID'].value_counts()
    normal_videos = video_counts[video_counts <= 10].index  # Chỉ giữ các video có <= 10 segment
    sponsor_df = sponsor_df[sponsor_df['videoID'].isin(normal_videos)]
    print(f"Sau khi loại bỏ video có quá nhiều segment: {sponsor_df.shape}, Thời gian: {time.time() - step_time:.2f}s")
    step_time = time.time()
    
    # 4. Tiêu chí chất lượng cao: ưu tiên theo locked > votes > reputation > timeSubmitted
    # Đối với mỗi video, chúng ta chỉ lấy các segment tốt nhất
    
    print("Bắt đầu lọc segment chất lượng cao...")
    
    # Cách tiếp cận hiệu quả: Gom nhóm theo videoID và chọn các segment tốt nhất
    sponsor_df['quality_score'] = (sponsor_df['locked'] * 1000 + 
                                 sponsor_df['votes'].clip(-10, 100) * 10 + 
                                 sponsor_df['reputation'].fillna(0))
    
    # Sắp xếp theo videoID và quality_score
    sponsor_df = sponsor_df.sort_values(['videoID', 'quality_score'], ascending=[True, False])
    
    # Chỉ chọn tối đa 3 segment chất lượng cao nhất cho mỗi video
    top_segments = sponsor_df.groupby('videoID').head(3)
    print(f"Sau khi chọn tối đa 3 segment chất lượng cao nhất: {top_segments.shape}, Thời gian: {time.time() - step_time:.2f}s")
    step_time = time.time()
    
    # Chỉ giữ các cột cần thiết
    top_segments = top_segments[['videoID', 'startTime', 'endTime', 'category', 'actionType']]
    
    return top_segments

# Thực hiện lọc tối ưu
filtered_sponsor_df = optimize_sponsorblock_filtering()

# Lưu dữ liệu đã lọc
filtered_sponsor_path = os.path.join(output_folder, 'filtered_sponsor_data.parquet')
filtered_sponsor_df.to_parquet(filtered_sponsor_path, index=False)
print(f"Đã lưu dữ liệu SponsorBlock đã lọc tại: {filtered_sponsor_path}")
print(f"Thời gian tổng cộng cho bước lọc: {time.time() - start_time:.2f} giây")

## 3. Xử lý hiệu quả dữ liệu Transcript

Tối ưu việc chuyển đổi dữ liệu transcript từ JSON sang DataFrame:
1. Sử dụng bộ lọc cho các video có trong dataset đã lọc
2. Sử dụng batch processing để giảm tải bộ nhớ

In [ ]:
def process_transcript_files_optimized(transcript_folder, sponsor_df, batch_size=1000):
    """
    Xử lý các file transcript JSON thành DataFrame với phương pháp xử lý theo batch
    """
    # Tạo set các videoID cần xử lý
    target_videos = set(sponsor_df['videoID'].unique())
    print(f"Số lượng video cần xử lý transcript: {len(target_videos)}")
    
    all_transcript_dfs = []
    file_count = 0
    batch_count = 0
    batch_segments = []
    
    # Đọc và xử lý từng file transcript
    start_time = time.time()
    for file_name in tqdm(os.listdir(transcript_folder), desc="Xử lý transcript"):
        if not file_name.endswith('.json'):
            continue
            
        # Lấy video_id từ tên file để kiểm tra nhanh
        video_id = file_name.split('.')[0]
        if video_id not in target_videos:
            continue
        
        file_count += 1
        file_path = os.path.join(transcript_folder, file_name)
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                transcript = json.load(f)
                
            # Kiểm tra xem transcript có dữ liệu không
            if transcript['status'] != 'success' or not transcript['transcripts']:
                continue
                
            # Lấy dữ liệu transcript tiếng Anh
            transcript_data = None
            for t in transcript['transcripts']:
                if t['language_code'] == 'en':
                    transcript_data = t['data']
                    break
                    
            if not transcript_data:
                continue
                
            # Xử lý từng đoạn trong transcript
            for segment in transcript_data:
                start_time_seg = segment['start']
                duration = segment['duration']
                end_time_seg = start_time_seg + duration
                
                batch_segments.append({
                    'video_id': video_id,
                    'start': start_time_seg,
                    'end': end_time_seg,
                    'duration': duration,
                    'text': segment['text']
                })
                
            # Xử lý theo batch để giảm tải bộ nhớ
            if len(batch_segments) >= batch_size:
                batch_count += 1
                all_transcript_dfs.append(pd.DataFrame(batch_segments))
                batch_segments = []
                print(f"Đã xử lý {file_count} files, {batch_count} batches, Thời gian: {time.time() - start_time:.2f}s")
                
        except Exception as e:
            print(f"Lỗi khi xử lý file {file_path}: {str(e)}")
            continue
            
    # Xử lý batch cuối cùng nếu còn
    if batch_segments:
        all_transcript_dfs.append(pd.DataFrame(batch_segments))
        
    # Ghép các DataFrame lại
    if all_transcript_dfs:
        final_transcript_df = pd.concat(all_transcript_dfs, ignore_index=True)
        return final_transcript_df
    else:
        return pd.DataFrame()

# Xử lý dữ liệu transcript
start_time = time.time()
transcript_df = process_transcript_files_optimized(transcript_folder, filtered_sponsor_df)

# Hiển thị thông tin
print(f"Số lượng dòng trong transcript DataFrame: {len(transcript_df)}")
print(f"Số lượng video duy nhất: {transcript_df['video_id'].nunique()}")

# Lưu transcript DataFrame
transcript_path = os.path.join(output_folder, 'processed_transcript_optimized.parquet')
transcript_df.to_parquet(transcript_path, index=False)
print(f"Đã lưu dữ liệu transcript đã xử lý tại: {transcript_path}")
print(f"Thời gian tổng cộng cho xử lý transcript: {time.time() - start_time:.2f} giây")

## 4. Gán nhãn nhanh cho transcript bằng phương pháp merge hiệu quả

Thay vì duyệt qua tất cả các hàng để gán nhãn, chúng ta sẽ:
1. Tối ưu hóa cấu trúc dữ liệu với dictionary
2. Sử dụng vectorized operations để gán nhãn nhanh

In [ ]:
def label_transcripts_fast(transcript_df, sponsor_df):
    """
    Gán nhãn cho transcript dựa trên dữ liệu SponsorBlock với phương pháp tối ưu
    """
    start_time = time.time()
    
    # Đổi tên cột để phù hợp khi merge
    sponsor_df_renamed = sponsor_df.rename(columns={'videoID': 'video_id'})
    
    print("Bắt đầu gán nhãn...")
    print(f"Số lượng bản ghi transcript: {len(transcript_df)}")
    print(f"Số lượng video trong transcript: {transcript_df['video_id'].nunique()}")
    print(f"Số lượng bản ghi sponsor: {len(sponsor_df_renamed)}")
    print(f"Số lượng video trong sponsor: {sponsor_df_renamed['video_id'].nunique()}")
    
    # Sử dụng phương pháp merge hiệu quả
    # Mỗi segment transcript sẽ được so với các segment sponsor của cùng một video
    result = pd.merge(
        transcript_df, 
        sponsor_df_renamed, 
        on='video_id', 
        how='left',
        suffixes=('', '_sponsor')
    )
    print(f"Kích thước sau khi merge: {result.shape}, Thời gian: {time.time() - start_time:.2f}s")
    
    # Gán nhãn sử dụng phép toán vector hóa
    result['overlap'] = (
        ((result['start'] >= result['startTime']) & (result['start'] < result['endTime'])) | 
        ((result['end'] > result['startTime']) & (result['end'] <= result['endTime'])) |
        ((result['start'] <= result['startTime']) & (result['end'] >= result['endTime']))
    )
    
    # Nhóm theo các cột của transcript để kiểm tra xem segment nào nằm trong sponsor
    # Một segment được đánh dấu là sponsor nếu nó có ít nhất một overlap = True
    transcript_with_overlap = result.groupby(
        ['video_id', 'start', 'end', 'duration', 'text']
    )['overlap'].any().reset_index()
    
    # Đổi tên cột overlap thành is_sponsor
    transcript_with_overlap = transcript_with_overlap.rename(columns={'overlap': 'is_sponsor'})
    
    # Chuyển boolean thành 0/1
    transcript_with_overlap['is_sponsor'] = transcript_with_overlap['is_sponsor'].astype(int)
    
    print(f"Đã hoàn thành gán nhãn, Thời gian: {time.time() - start_time:.2f}s")
    
    return transcript_with_overlap

# Gán nhãn cho transcript
start_time = time.time()
labeled_transcript_df = label_transcripts_fast(transcript_df, filtered_sponsor_df)

# Hiển thị thông tin
print(f"Tổng số dòng: {len(labeled_transcript_df)}")
print(f"Số dòng được gán nhãn sponsor (is_sponsor=1): {labeled_transcript_df['is_sponsor'].sum()}")
print(f"Tỷ lệ sponsor: {labeled_transcript_df['is_sponsor'].mean() * 100:.2f}%")

# Lưu DataFrame đã gán nhãn
labeled_path = os.path.join(output_folder, 'transcript_segments_labeled_optimized.parquet')
labeled_transcript_df.to_parquet(labeled_path, index=False)
print(f"Đã lưu dữ liệu transcript đã gán nhãn tại: {labeled_path}")
print(f"Thời gian tổng cộng cho gán nhãn: {time.time() - start_time:.2f} giây")

## 5. Phân tích và kiểm tra dữ liệu đã gán nhãn

In [ ]:
# Phân tích dữ liệu đã gán nhãn
print("===== THỐNG KÊ DỮ LIỆU ĐÃ GÁN NHÃN =====")
print(f"Số lượng video: {labeled_transcript_df['video_id'].nunique()}")
print(f"Số lượng đoạn text: {len(labeled_transcript_df)}")
print(f"Số lượng đoạn sponsor: {labeled_transcript_df['is_sponsor'].sum()}")
print(f"Tỷ lệ sponsor: {labeled_transcript_df['is_sponsor'].mean() * 100:.2f}%")

# Phân tích theo nhãn
sponsor_segments = labeled_transcript_df[labeled_transcript_df['is_sponsor'] == 1]
non_sponsor_segments = labeled_transcript_df[labeled_transcript_df['is_sponsor'] == 0]

print("\n===== THỐNG KÊ THEO NHÃN =====")
print("Đoạn sponsor:")
print(f"- Số lượng: {len(sponsor_segments)}")
print(f"- Thời lượng trung bình: {sponsor_segments['duration'].mean():.2f} giây")

print("\nĐoạn không phải sponsor:")
print(f"- Số lượng: {len(non_sponsor_segments)}")
print(f"- Thời lượng trung bình: {non_sponsor_segments['duration'].mean():.2f} giây")

## 6. Tạo tập dữ liệu cuối cùng cân bằng

Tạo một tập dữ liệu cân bằng và nhỏ gọn cho việc huấn luyện mô hình

In [ ]:
def create_final_dataset(labeled_df, output_path):
    """
    Tạo tập dữ liệu cuối cùng cân bằng và có kích thước hợp lý
    """
    # Lấy toàn bộ segment sponsor
    sponsor_segments = labeled_df[labeled_df['is_sponsor'] == 1]
    
    # Lấy mẫu cân bằng từ non-sponsor segments
    non_sponsor_segments = labeled_df[labeled_df['is_sponsor'] == 0].sample(
        n=min(len(sponsor_segments) * 3, len(labeled_df[labeled_df['is_sponsor'] == 0])),
        random_state=42
    )
    
    # Ghép lại
    balanced_df = pd.concat([sponsor_segments, non_sponsor_segments], ignore_index=True)
    
    # Shuffle dataset
    balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Lưu lại
    balanced_df.to_parquet(output_path, index=False)
    
    return balanced_df

# Tạo tập dữ liệu cuối cùng
final_dataset_path = os.path.join(output_folder, 'final_balanced_dataset.parquet')
final_df = create_final_dataset(labeled_transcript_df, final_dataset_path)

print("\n===== THỐNG KÊ TẬP DỮ LIỆU CUỐI CÙNG =====")
print(f"Số lượng video: {final_df['video_id'].nunique()}")
print(f"Tổng số dòng: {len(final_df)}")
print(f"Số dòng sponsor: {final_df['is_sponsor'].sum()} ({final_df['is_sponsor'].mean() * 100:.2f}%)")
print(f"Số dòng không phải sponsor: {len(final_df) - final_df['is_sponsor'].sum()} ({(1 - final_df['is_sponsor'].mean()) * 100:.2f}%)")
print(f"Đã lưu tập dữ liệu cuối cùng tại: {final_dataset_path}")

## Tóm tắt quá trình xử lý và so sánh với phương pháp trước

### Tối ưu hóa đã thực hiện:

1. **Lọc dữ liệu thông minh**:
   - Chỉ lấy các segment chất lượng cao dựa trên tiêu chí của SponsorBlock
   - Loại bỏ sớm các segment không cần thiết
   - Giới hạn số segment cho mỗi video (tối đa 3 segment chất lượng cao nhất)

2. **Xử lý dữ liệu theo batch**:
   - Chỉ xử lý transcript cho các video đã lọc (thay vì tất cả 7 triệu video)
   - Xử lý theo batch để giảm tải bộ nhớ

3. **Gán nhãn hiệu quả**:
   - Sử dụng merge thay vì duyệt qua từng dòng 
   - Áp dụng phép toán vector hóa cho việc kiểm tra overlap

4. **Tạo tập dữ liệu cân bằng**:
   - Cân bằng tỷ lệ giữa segment sponsor và không phải sponsor (1:3)
   - Dataset có kích thước hợp lý cho huấn luyện mô hình

### Ước tính tăng tốc:
- Phương pháp cũ: > 800 giờ cho 7 triệu video
- Phương pháp mới: ước tính < 8 giờ (giảm ~99% thời gian)

### Nhược điểm tiềm tàng:
- Có thể bỏ sót một số segment có chất lượng thấp
- Tập dữ liệu cuối cùng nhỏ hơn nhưng cân bằng và chất lượng cao hơn

In [ ]:
"""
transcript format:
{
    "video_id": "4pa9O5cr398",
    "timestamp": "2025-04-06T19:14:11.761808",
    "status": "success",
    "error_type": null,
    "error_message": null,
    "transcripts": [
        {
            "language": "English (auto-generated)",
            "language_code": "en",
            "is_generated": true,
            "data": [
                {
                    "text": "hello",
                    "start": 9.28,
                    "duration": 2.88
                },
                {
...
                {
                    "text": "you",
                    "start": 723.12,
                    "duration": 2.08
                }
            ]
        }
    ]
}
"""

'\ntranscript format:\n{\n    "video_id": "4pa9O5cr398",\n    "timestamp": "2025-04-06T19:14:11.761808",\n    "status": "success",\n    "error_type": null,\n    "error_message": null,\n    "transcripts": [\n        {\n            "language": "English (auto-generated)",\n            "language_code": "en",\n            "is_generated": true,\n            "data": [\n                {\n                    "text": "hello",\n                    "start": 9.28,\n                    "duration": 2.88\n                },\n                {\n...\n                {\n                    "text": "you",\n                    "start": 723.12,\n                    "duration": 2.08\n                }\n            ]\n        }\n    ]\n}\n'

In [ ]:
import pandas as pd
import os
import json
import re
import numpy as np
from tqdm import tqdm
from datetime import datetime
from typing import List, Dict, Any, Tuple
from collections import defaultdict
import time

# Đường dẫn dữ liệu
sponsor_dataset = r"C:\Users\caotr\Downloads\transcript_data\sponsorTimes.parquet"
video_info_dataset = r"C:\Users\caotr\Downloads\transcript_data\videoInfo.csv"
transcript_folder = r"C:\Users\caotr\Downloads\transcript_data\transcripts"

# Đường dẫn lưu kết quả
output_folder = r"C:\Users\caotr\Documents\Github\Nh-m-4---IE403.KTDLTTXH\data_raw"
os.makedirs(output_folder, exist_ok=True)

# Đặt số lượng luồng cho pandas để tối ưu xử lý
n_threads = os.cpu_count() - 1
pd.options.mode.chained_assignment = None  # Tắt cảnh báo SettingWithCopyWarning

## 1. Hiểu về cách SponsorBlock hiển thị segment

SponsorBlock sẽ hiển thị segment trên video dựa trên các tiêu chí sau:
1. **Segment không bị ẩn** (hidden=0, shadowHidden=0)
2. **Segment có điểm votes cao**
3. **Segment được khóa** (locked=1) có ưu tiên cao
4. **Segment có reputation cao**
5. **Segment có thời gian submit gần đây nhất** trong trường hợp chồng chéo

Dựa vào cách hoạt động của SponsorBlock, chúng ta sẽ lọc dữ liệu theo các tiêu chí trên để tăng tốc độ xử lý mà vẫn đảm bảo chất lượng.